In [3]:
! pip install gradio

In [4]:
import gradio as gr
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
from prophet import Prophet
import numpy as np

def get_nearest_date(date, date_series):
    return date_series[np.abs(date_series - date).argmin()]

def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)

        # Get historical data for the past 10 years (if available)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=3650)  # Approximately 10 years
        hist = stock.history(start=start_date, end=end_date)

        if hist.empty:
            raise ValueError(f"No data available for symbol {symbol}")

        # Get the current price (last available closing price)
        current_price = hist['Close'].iloc[-1]

        # Get the most recent opening and closing prices
        latest_data = hist.iloc[-1]
        opening_price = latest_data['Open']
        closing_price = latest_data['Close']

        # Calculate max and min prices
        max_price = hist['High'].max()
        min_price = hist['Low'].min()

        # Prepare data for Prophet
        prophet_data = hist.reset_index()[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
        # Remove timezone information
        prophet_data['ds'] = prophet_data['ds'].dt.tz_localize(None)

        # Create and fit the Prophet model
        model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        model.fit(prophet_data)

        # Generate future dates for prediction (10 years)
        future_dates = model.make_future_dataframe(periods=3650)

        # Make predictions
        forecast = model.predict(future_dates)

        # Get predictions for 1, 5, and 10 years
        pred_dates = [end_date + timedelta(days=365), end_date + timedelta(days=1825), end_date + timedelta(days=3650)]
        pred_labels = ["1 year", "5 years", "10 years"]
        predictions = []

        for date in pred_dates:
            nearest_date = get_nearest_date(date, forecast['ds'])
            pred_price = forecast[forecast['ds'] == nearest_date]['yhat'].values[0]
            predictions.append(pred_price)

        # Create plots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

        # Historical price plot
        ax1.plot(hist.index, hist['Close'])
        ax1.set_title(f"{symbol} Stock Price - Historical Data")
        ax1.set_xlabel("Date")
        ax1.set_ylabel("Price (USD)")
        ax1.grid(True)

        # Forecast plot
        model.plot(forecast, ax=ax2)
        ax2.set_title(f"{symbol} Stock Price - 10 Year Forecast")
        ax2.set_xlabel("Date")
        ax2.set_ylabel("Price (USD)")

        plt.tight_layout()

        return (
            f"Current price of {symbol}: ${current_price:.2f}",
            f"Opening price: ${opening_price:.2f}",
            f"Closing price: ${closing_price:.2f}",
            f"Max price: ${max_price:.2f}",
            f"Min price: ${min_price:.2f}",
            f"Predicted price (1 year): ${predictions[0]:.2f}",
            f"Predicted price (5 years): ${predictions[1]:.2f}",
            f"Predicted price (10 years): ${predictions[2]:.2f}",
            fig
        )
    except Exception as e:
        return str(e), "", "", "", "", "", "", "", None

# Create the Gradio interface
app = gr.Interface(
    fn=get_stock_info,
    inputs=gr.Textbox(label="Enter stock symbol (e.g., AAPL, GOOGL)"),
    outputs=[
        gr.Textbox(label="Current Price"),
        gr.Textbox(label="Opening Price"),
        gr.Textbox(label="Closing Price"),
        gr.Textbox(label="Max Price"),
        gr.Textbox(label="Min Price"),
        gr.Textbox(label="Predicted Price (1 year)"),
        gr.Textbox(label="Predicted Price (5 years)"),
        gr.Textbox(label="Predicted Price (10 years)"),
        gr.Plot(label="Price Chart and Forecast")
    ],
    title="Stock Price Viewer and Long-Term Predictor",
    description="Enter a stock symbol to view its current, opening, closing, max, and min prices, along with a price chart and 1, 5, and 10-year forecasts."
)

# Launch the app
app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b00d17d3ab418efa2f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
